In [1]:
!curl https://www.gutenberg.org/files/1661/1661-0.txt -o book.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  3  593k    3 24030    0     0  22689      0  0:00:26  0:00:01  0:00:25 22863
 72  593k   72  428k    0     0   210k      0  0:00:02  0:00:02 --:--:--  211k
100  593k  100  593k    0     0   264k      0  0:00:02  0:00:02 --:--:--  265k


In [2]:
text = open('book.txt', 'r', encoding = 'utf-8').read()
text = text.lower()

In [3]:
sentences = text.split('\n')

In [4]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
tokenizer = Tokenizer(oov_token = '<UNK>')
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1
vocab_size

8923

In [6]:
sequences = tokenizer.texts_to_sequences(sentences)

In [10]:
input_sequences = []
for sequence in sequences:
    for i in range(len(sequence)):
        n_gram_seq = sequence[:i+1]
        input_sequences.append(n_gram_seq)

In [16]:
max_seq_len = max(len(seq) for seq in input_sequences)
max_seq_len

20

In [17]:
padded_seq = pad_sequences(input_sequences, max_seq_len)

In [19]:
import numpy as np

In [20]:
padded_seq = np.array(padded_seq)

In [34]:
x = padded_seq[:,:-1]
y = padded_seq[:,-1]

In [35]:
x.shape, y.shape

((111152, 19), (111152,))

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [38]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_shape=(max_seq_len-1,)))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(vocab_size, activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ (None, 19, 100)             │         892,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_5 (Bidirectional)      │ (None, 512)                 │         731,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 8923)                │       4,577,499 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,200,935 (23.65 MB)

 Trainable params: 6,200,935 (23.65 MB)

 Non-trainable params: 0 (0.00 B)

In [39]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='accuracy',min_delta = 0.01)

In [40]:
model.fit(x,y,epochs=50,verbose=1,batch_size=512,callbacks=[es])

Epoch 1/50
 60/218 ━━━━━━━━━━━━━━━━━━━━ 2:35 984ms/step - accuracy: 0.0397 - loss: 7.3746 

KeyboardInterrupt: 